In [84]:
import json
import pandas as pd
from poprogress import simple_progress as simp
import random

In [5]:
file_path = "../data/conll-balanced.json"

with open(file_path) as f:
    text = json.load(f)
    print("-"*30)
    print("numbers of sentences: ", len(text))
    print("-"*30)

------------------------------
numbers of sentences:  21363
------------------------------


In [207]:
def get_df(data):

    df = pd.DataFrame()
    for i,(sentence, annotation) in simp(enumerate(data)):
        # tags, words = get_sent_label(sentence, annotation)
        tag_dic = get_sent_label(sentence, annotation)
        # for j,x in enumerate(tag_dic):
        #     dataset = {"sent_id": i, "raw_sentence": sentence, "word_id": j, "word": x, "tag": tag_dic[x]}
        #     # print(dataset)
            
        #     df1 = pd.DataFrame(dataset, index=[0])
        #     df = pd.concat([df, df1], ignore_index=True)
        dataset = {"sent_id": i, 
                #    "raw_sentence": sentence, 
                   "word_id": list(range(len(tag_dic))), 
                   "word": list(tag_dic), 
                   "tag": list(tag_dic.values())}
        # print(dataset)
        
        df1 = pd.DataFrame(dataset)
        df = pd.concat([df, df1], ignore_index=True)

    # print(df)
    return df

def get_sent_label(sent, anns):
    '''
    BIO
    param:
        - sent: sentence
        - ann : annotation of the sentence
    '''
    words = get_words(sent)

    tag_dic = get_tag_dic(anns, words)
        
    # id_to_tag = {}
    # tag_val = tag_dic.values()
    # for i,x in enumerate(tag_val):
    #     id_to_tag[i] = x
        
    # id_to_word = {}
    # for i,x in enumerate(list(tag_dic)):
    #     id_to_word[i] = x
        
    # print(id_to_tag)
    # print(id_to_word)
    # return id_to_tag, id_to_word
    return tag_dic

def get_words(sent):
    '''
    Clear punctuation
    '''
    words_list = sent.split()
    # eli = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
    words = []
    for word in words_list:
        words.append(word)
        # if word not in eli:
            # words.append(word)
    # print(words)
    return words

def get_tag_dic(anns, words):
    dic = dict()
    for word in words:
        dic[word] = 'O'
        
    for ann in anns:
        start = ann["start"]
        end = ann["end"]
        label = ann["label"]
        text = ann["text"]
        text_word = text.split()
        
        for i,x in enumerate(text_word):
            if x in words:
                if i == 0:
                    dic[x] = 'B-' + label
                else:
                    dic[x] = 'I-' + label
    # print(dic)
    
    return dic


In [208]:
def data_split(full_list, ratio, shuffle=False):

    n_total = len(full_list)
    offset = int(n_total * ratio)
    
    if n_total == 0 or offset < 1:
        return [], full_list

    if shuffle:
        random.shuffle(full_list)
    test_set = full_list[:offset]
    train_set = full_list[offset:]
    return test_set, train_set
 
test_data, train_data = data_split(text, ratio=0.2, shuffle=True)
print(len(test_data))
print(len(train_data))


4272
17091


In [209]:
test_df = get_df(test_data)
print(test_df.columns)
test_df.to_csv("test.csv", index=None)

0it [00:00, ?it/s]

4272it [00:15, 284.63it/s]


Index(['sent_id', 'word_id', 'word', 'tag'], dtype='object')


In [210]:
train_df = get_df(train_data)
train_df.to_csv("train.csv", index=None)

17091it [02:00, 141.59it/s]
